### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [3]:
# cell for work
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [4]:
# cell for work
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [5]:
# more work
print(movies.movie.nunique())
print(movies.genre.nunique())
print(movies.movie_id.nunique())
print(movies.shape)

35416
2736
35479
(35479, 3)


In [6]:
# and still more
print(reviews.movie_id.nunique())
print(reviews.user_id.nunique())
print(reviews.shape)
print(len(reviews))
print(reviews.rating.mean())
print(reviews.rating.min())
print(reviews.rating.max())
print(reviews.rating.isna().sum(axis=0))

35479
67353
(863866, 4)
863866
7.31587769399
0
10
0


In [7]:
# and even more - open additional cells as necessary

In [8]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': movies.movie_id.nunique(),
'The number of ratings in the dataset': len(reviews),
'The number of different genres': movies.genre.nunique(),
'The number of unique users in the dataset': reviews.user_id.nunique(),
'The number missing ratings in the reviews dataset': reviews.rating.isna().sum(axis=0),
'The average rating given across all ratings': reviews.rating.mean(),
'The minimum rating given across all ratings': reviews.rating.min(),
'The maximum rating given across all ratings': reviews.rating.max(),
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [9]:
# cell for work
movies[['movie','year']] = movies['movie'].str.split('(',expand=True)
movies.head()

,movie_id,movie,genre,year
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1894)
1,0000010,La sortie des usines Lumière,Documentary|Short,1895)
2,0000012,The Arrival of a Train,Documentary|Short,1896)
3,25,The Oxford and Cambridge University Boat Race,NaN,1895)
4,0000091,Le manoir du diable,Short|Horror,1896)


In [10]:
# and another
movies['year'] = movies['year'].map(lambda x: x.rstrip(')'))
movies.head()

,movie_id,movie,genre,year
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1894
1,0000010,La sortie des usines Lumière,Documentary|Short,1895
2,0000012,The Arrival of a Train,Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race,NaN,1895
4,0000091,Le manoir du diable,Short|Horror,1896


In [11]:
# and a few more below, which you can use as necessary
y = movies['year'].astype(int)
movies["1800's"] = np.where(y<1900, 1, 0)
movies["1900's"] = np.where((y>=1900) & (y<2000), 1, 0)
movies["2000's"] = np.where(y>=2000, 1, 0)

In [12]:
movies.head()

,movie_id,movie,genre,year,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière,Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train,Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race,NaN,1895,1,0,0
4,0000091,Le manoir du diable,Short|Horror,1896,1,0,0


In [15]:
genres = []
for val in movies['genre']:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass

In [16]:
def split_genres(val):
    try:
        if val.find(g) > -1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

In [18]:
genres = set(genres)
print(len(genres))
genres

28


{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [19]:
for g in genres:
    movies[g] = movies['genre'].apply(split_genres)

In [20]:
movies.head()

,movie_id,movie,genre,year,1800's,1900's,2000's,Documentary,Short,Horror,...,Thriller,Animation,Music,Musical,Film-Noir,Game-Show,Talk-Show,Adult,News,Reality-TV
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1894,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière,Documentary|Short,1895,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train,Documentary|Short,1896,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable,Short|Horror,1896,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [21]:
movies.iloc[1]

movie_id                             0000010
movie          La sortie des usines Lumière 
genre                      Documentary|Short
year                                    1895
1800's                                     1
1900's                                     0
2000's                                     0
Documentary                                1
Short                                      1
Horror                                     0
Comedy                                     0
Action                                     0
Adventure                                  0
Fantasy                                    0
Sci-Fi                                     0
Crime                                      0
Western                                    0
Drama                                      0
Romance                                    0
History                                    0
Family                                     0
War                                        0
Sport     

In [ ]:
# Get one hot encoding of columns 'vehicleType'
one_hot = pd.get_dummies(movies['genre'])
# Drop column as it is now encoded
#data_df = data_df.drop('vehicleType',axis = 1)
# Join the encoded df
data_df = data_df.join(one_hot)
data_df 

In [ ]:
reviews_new, movies_new = t.show_clean_dataframes()